# Stacking different models together

In [1]:
# collecting randomforest, neural network, logistic regress
import models
import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd

# Stacking classifiers

In [4]:
df = preprocessing.column_selection("data/train.csv")
X = df.drop(columns="Survived", axis=1)
Y = df["Survived"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# rescale data in trainnig stage (decrease sensitivity of the model)            make sure inputs are in the range of activision function (so MLP could learn from features)
sc = StandardScaler()
scaler = sc.fit(X)
trainX_scaled = scaler.transform(X_train)
testX_scaled = scaler.transform(X_test)

nn = models.neuralNetwork(X_train, Y_train)
logReg = models.logRegression(X_train, Y_train)
forest = models.randforestclassf(X_train, Y_train)

#estimators = [('mlp', MLPClassifier(hidden_layer_sizes=(6, 2), max_iter=5000, activation="relu", solver="adam")), ('log', LogisticRegression(penalty="l2", max_iter=5000))]
estimators = [('mlp', nn), ('log', logReg), ('rf', forest)]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(penalty="l2", max_iter=5000))

pol = PolynomialFeatures(2)

clf.fit(pol.fit_transform(trainX_scaled), Y_train).score(pol.fit_transform(testX_scaled), Y_test)


0.8033707865168539

In [14]:
df2 = preprocessing.column_selection("data/test.csv")
y_pred = clf.predict(pol.fit_transform(df2))
y_pred = pd.DataFrame(y_pred)
y_pred.columns = ["Survived"]
y_pred.to_csv("submission.csv")

# Bagging

In [ ]:
# separate train data into several subsets and fit them into different models 
BaggingClassifier()